In [ ]:
import os
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 1. Load data
df = pd.read_csv(r"../Datasets/jobs_cleaned.csv")
# Loại bỏ dòng trống để tránh lỗi mảng 0 sample
df = df.dropna(subset=['e5_input']).reset_index(drop=True)
docs = df['e5_input'].tolist()

# 2. Embedding với E5 (Dùng bản multilingual cho Anh-Việt)
embedding_model = SentenceTransformer('intfloat/multilingual-e5-large')
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# 3. Cấu hình các thành phần (Fix lỗi ensure_all_finite bằng cách khởi tạo mới)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)

Batches: 100%|██████████| 105/105 [47:01<00:00, 26.87s/it]


In [ ]:
# Lưu models để sử dụng lại sau này
embedding_model.save(r"../Models/embedding_model")
embedding_path = r"../Models/embedding_model/embeddings.npy"
np.save(embedding_path, embeddings)

# Chạy lại 

In [23]:
import os
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv(r"../Datasets/jobs_cleaned.csv")
# Loại bỏ dòng trống để tránh lỗi mảng 0 sample
df = df.dropna(subset=['e5_input']).reset_index(drop=True)
docs = df['e5_input'].tolist()

print(f"Số lượng văn bản (docs): {len(docs)}")


Số lượng văn bản (docs): 3358


In [24]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)
embedding_model = SentenceTransformer(r"../Models/embedding_model")
embeddings = embedding_model.encode(docs, show_progress_bar=True)
embedding_path = r"../Models/embedding_model/embeddings.npy"
np.save(embedding_path, embeddings)

Batches: 100%|██████████| 105/105 [41:11<00:00, 23.53s/it]


In [25]:
# 4. Khởi tạo và chạy BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    nr_topics="auto"
)

topics, probs = topic_model.fit_transform(docs, embeddings)

# # 5. Tính toán phân bổ theo chuyên môn (Để dùng cho đoạn code dưới)
# topics_per_class = topic_model.topics_per_class(docs, classes=classes)

# # 6. Trực quan hóa
# fig_topics = topic_model.visualize_topics()
# fig_hierarchy = topic_model.visualize_hierarchy()
# fig_barchart = topic_model.visualize_barchart(top_n_topics=10)
# fig_class = topic_model.visualize_topics_per_class(topics_per_class)

# Lưu kết quả
topic_model.save(r"../Models/job_topic_model")
# fig_barchart.write_html(r"../Figures/topic_keywords.html")
# fig_class.write_html(r"../Figures/topics_by_job_role.html")


2026-01-08 15:23:02,779 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [26]:
# 1. Gán Topic ID vào DataFrame
df['Topic_ID'] = topics

# 2. Tạo hàm lấy 5 từ khóa cho mỗi Topic ID
# Lấy bảng thông tin chung của các topic
all_topics_info = topic_model.get_topics()

def get_top_5_words(topic_id):
    if topic_id == -1:
        return "Nhiễu/Không xác định"
    # Lấy danh sách từ khóa và chọn 5 từ đầu tiên
    words = all_topics_info[topic_id]
    top_5 = ", ".join([w[0] for w in words[:5]])
    return top_5

# 3. Tạo cột mới 'Topic_Keywords' dựa trên Topic_ID
# Tạo một từ điển map cho nhanh
topic_map = {topic_id: get_top_5_words(topic_id) for topic_id in set(topics)}
df['Topic_Keywords'] = df['Topic_ID'].map(topic_map)

# 4. Lưu lại dataset mới
df.to_csv(r"../Datasets/jobs_with_topics.csv", index=False, encoding="utf-8-sig")

print(" Đã gán Topic ID và 5 từ khóa vào dataset.")
print(df[['Tiêu đề', 'Topic_ID', 'Topic_Keywords']].head())

 Đã gán Topic ID và 5 từ khóa vào dataset.
               Tiêu đề  Topic_ID                      Topic_Keywords
0         Data Analyst         0  data, công, việc, experience, liệu
1         Data Analyst        -1                Nhiễu/Không xác định
2         Data Analyst         0  data, công, việc, experience, liệu
3         Data Analyst         0  data, công, việc, experience, liệu
4  Senior Data Analyst         0  data, công, việc, experience, liệu


In [27]:
# 1. Lấy bảng thông tin tổng quát về các topic
topic_info = topic_model.get_topic_info()

# 2. Lấy top 15 topic (loại bỏ Topic -1 vì đây là nhóm nhiễu/outliers)
top_15_topics = topic_info[topic_info["Topic"] != -1].head(15)

print(f"{'ID':<5} | {'Số lượng':<10} | {'Từ khóa đặc trưng cho Topic'}")
print("-" * 80)

# 3. Duyệt qua từng topic để in chi tiết
for probs, row in top_15_topics.iterrows():
    topic_id = row["Topic"]
    count = row["Count"]
    
    # Lấy danh sách từ khóa của topic này
    words_data = topic_model.get_topic(topic_id)
    
    if words_data:
        # Lấy 7 từ khóa đầu tiên để mô tả rõ nét hơn
        description = ", ".join([word[0] for word in words_data[:7]])
    else:
        description = "Không có dữ liệu"
        
    print(f"{topic_id:<5} | {count:<10} | {description}")

# --- Mẹo: Nếu bạn muốn lưu danh sách này ra file Excel để báo cáo ---
# top_15_topics.to_excel("../Datasets/top_15_topics_summary.xlsx", index=False)

ID    | Số lượng   | Từ khóa đặc trưng cho Topic
--------------------------------------------------------------------------------
0     | 1957       | data, công, việc, experience, liệu, dữ, ai
1     | 82         | khách, hàng, tư, vấn, học, chăm, không
2     | 50         | binance, blockchain, trading, ecosystem, arrangement, worlds, web3
3     | 38         | community, exam, assessment, media, complete, content, internet
4     | 35         | data, publicis, consulting, team, experience, candidates, equal
5     | 33         | mb, bank, phone, directly, applicants, recruitment, application
6     | 28         | et, description, group, technical, vietnam, colleagues, database
7     | 26         | asia, branches, united, risk, bank, offices, banking
8     | 23         | vietnam, labor, contact, development, skills, performance, experience
9     | 22         | crossian, marketing, campaign, performance, lifetime, campaigns, meta
10    | 21         | microsoft, solutions, transformation, se

In [28]:
# 1. Lấy toàn bộ thông tin topic
all_topic_info = topic_model.get_topic_info()

# 2. Loại bỏ topic -1 (outlier) để xem các chủ đề chính thống
filtered_info = all_topic_info[all_topic_info["Topic"] != -1]

print(f"Tổng số lượng chủ đề tìm thấy: {len(filtered_info)}")
print("-" * 100)
print(f"{'ID':<5} | {'Số lượng':<10} | {'Từ khóa mô tả chủ đề'}")
print("-" * 100)

for probs, row in filtered_info.iterrows():
    t_id = row["Topic"]
    t_count = row["Count"]
    
    # Lấy 10 từ khóa để có cái nhìn chi tiết nhất cho mỗi topic
    words = topic_model.get_topic(t_id)
    t_desc = ", ".join([w[0] for w in words[:10]])
    
    print(f"{t_id:<5} | {t_count:<10} | {t_desc}")

Tổng số lượng chủ đề tìm thấy: 23
----------------------------------------------------------------------------------------------------
ID    | Số lượng   | Từ khóa mô tả chủ đề
----------------------------------------------------------------------------------------------------
0     | 1957       | data, công, việc, experience, liệu, dữ, ai, business, kinh, phân
1     | 82         | khách, hàng, tư, vấn, học, chăm, không, công, kinh, việc
2     | 50         | binance, blockchain, trading, ecosystem, arrangement, worlds, web3, hiring, tools, financial
3     | 38         | community, exam, assessment, media, complete, content, internet, qualification, project, relevance
4     | 35         | data, publicis, consulting, team, experience, candidates, equal, possible, 60, solutions
5     | 33         | mb, bank, phone, directly, applicants, recruitment, application, oracle, email, information
6     | 28         | et, description, group, technical, vietnam, colleagues, database, people, 26, co